<a href="https://colab.research.google.com/github/edmangog/pygoogletrans/blob/master/pygoogletrans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
pip install googletrans==4.0.0rc1

In [ ]:
pip install fasttext

In [ ]:
from googletrans import Translator
import pandas as pd
import fasttext

translator = Translator(raise_exception=True)

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
!gunzip cc.en.300.bin.gz
model = fasttext.load_model('lid.176.bin')

# Main code

In [ ]:
df = pd.read_csv('sample.csv', header=0, index_col=0, encoding='utf-8-sig')

Create lookup table

In [ ]:
dict = {
  'text':df.drop_duplicates(subset=['text'])['text'],
  'first_lang':[],
  'first_lang_score':[],
  'second_lang':[],
  'second_lang_score':[],
  'translated':[]
}

Language detection and translation

In [ ]:
import tqdm
progress = tqdm.tqdm(total=len(dict['text']))

for tweet in dict['text']:
  result = model.predict(tweet,k=2)
  first_lang = result[0][0]
  first_lang_score = result[1][0]
  second_lang = result[0][1]
  second_lang_score = result[1][1]

  if first_lang != '__label__en' and first_lang_score < 66:
    dict['translated'].append(translator.translate(tweet, dest='en').text)
  else:
    dict['translated'].append('null')

  dict['first_lang'].append(first_lang)
  dict['first_lang_score'].append(first_lang_score)
  dict['second_lang'].append(second_lang)
  dict['second_lang_score'].append(second_lang_score)

  progress.update(1)



Lookup and export

In [ ]:
lookup_df = pd.DataFrame.from_dict(dict)

# df.merge(lookup_df, on='text', how='left')
df2 = df.merge(lookup_df, on='text', how='left')
df2.to_csv('result.csv', encoding='utf-8-sig')